In [38]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
import numpy as np
train_x=np.array([[0.129779 ,0.352941 ,0.245902 ,0.527273 ,0.666667 ,0.002290 ,0.000000 ,0.0],  
        [0.148893 ,0.367647 ,0.245902 ,0.527273 ,0.666667 ,0.003811 ,0.000000 ,0.0],
        [0.159960 ,0.426471 ,0.229508 ,0.545454 ,0.666667 ,0.005332 ,0.000000 ,0.0],
        [0.182093 ,0.485294 ,0.229508 ,0.563637 ,0.666667 ,0.008391 ,0.037037 ,0.0],
        [0.138833 ,0.485294 ,0.229508 ,0.563637 ,0.666667 ,0.009912 ,0.074074 ,0.0]])
train_y=np.array([0.148893 ,0.159960 ,0.182093 ,0.138833 ,0.109658])

In [40]:
train_x

array([[0.129779, 0.352941, 0.245902, 0.527273, 0.666667, 0.00229 ,
        0.      , 0.      ],
       [0.148893, 0.367647, 0.245902, 0.527273, 0.666667, 0.003811,
        0.      , 0.      ],
       [0.15996 , 0.426471, 0.229508, 0.545454, 0.666667, 0.005332,
        0.      , 0.      ],
       [0.182093, 0.485294, 0.229508, 0.563637, 0.666667, 0.008391,
        0.037037, 0.      ],
       [0.138833, 0.485294, 0.229508, 0.563637, 0.666667, 0.009912,
        0.074074, 0.      ]])

In [41]:
train_x=torch.from_numpy(train_x)

In [42]:
train_x=

tensor([[0.1298, 0.3529, 0.2459, 0.5273, 0.6667, 0.0023, 0.0000, 0.0000],
        [0.1489, 0.3676, 0.2459, 0.5273, 0.6667, 0.0038, 0.0000, 0.0000],
        [0.1600, 0.4265, 0.2295, 0.5455, 0.6667, 0.0053, 0.0000, 0.0000],
        [0.1821, 0.4853, 0.2295, 0.5636, 0.6667, 0.0084, 0.0370, 0.0000],
        [0.1388, 0.4853, 0.2295, 0.5636, 0.6667, 0.0099, 0.0741, 0.0000]],
       dtype=torch.float64)

In [43]:
train_y

array([0.148893, 0.15996 , 0.182093, 0.138833, 0.109658])

In [44]:
train_y=torch.from_numpy(train_y)

In [45]:
train_y

tensor([0.1489, 0.1600, 0.1821, 0.1388, 0.1097], dtype=torch.float64)

In [46]:
train_x = Variable(train_x, requires_grad=True)
train_y = Variable(train_y, requires_grad=True)
train_x

tensor([[0.1298, 0.3529, 0.2459, 0.5273, 0.6667, 0.0023, 0.0000, 0.0000],
        [0.1489, 0.3676, 0.2459, 0.5273, 0.6667, 0.0038, 0.0000, 0.0000],
        [0.1600, 0.4265, 0.2295, 0.5455, 0.6667, 0.0053, 0.0000, 0.0000],
        [0.1821, 0.4853, 0.2295, 0.5636, 0.6667, 0.0084, 0.0370, 0.0000],
        [0.1388, 0.4853, 0.2295, 0.5636, 0.6667, 0.0099, 0.0741, 0.0000]],
       dtype=torch.float64, requires_grad=True)

In [47]:
train_y

tensor([0.1489, 0.1600, 0.1821, 0.1388, 0.1097], dtype=torch.float64,
       requires_grad=True)

In [48]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        # the hidden_size is 51
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 1)

    def forward(self, input, future=0):
        outputs = []
        # both the input(h_t, c_t) and output(h_t2, c_t2) are initialized to zeros
        h_t = Variable(
            torch.zeros(input.size(0), 51), requires_grad=False)
        c_t = Variable(
            torch.zeros(input.size(0), 51), requires_grad=False)
        h_t2 = Variable(
            torch.zeros(input.size(0), 1), requires_grad=False)
        c_t2 = Variable(
            torch.zeros(input.size(0), 1), requires_grad=False)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(c_t, (h_t2, c_t2))
            outputs += [c_t2]

        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

In [49]:
# build the model
seq = Sequence()
criterion = nn.MSELoss()
# use LBFGS as optimizer since we can load the whole data to train
optimizer = torch.optim.LBFGS(seq.parameters())

In [34]:
loss_list = []
test_loss_list = []
epoch_num = 50

# begin to train
for epoch in range(epoch_num):
    print('epoch : ', epoch)

    def closure():
        optimizer.zero_grad()
        out = seq(train_x)
        loss = criterion(out, train_y)
#         print('loss:', loss.data.numpy()[0])
        loss_list.append(loss.data.numpy()[0])
        loss.backward()
        return loss

    optimizer.step(closure)

    pred = seq(test_X)
    loss = criterion(pred, test_y)
#     print('test loss:', loss.data.numpy()[0])
    test_loss_list.append(loss.data.numpy()[0])

    y = pred.data.numpy()

epoch :  0


RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'